In [39]:
import numpy as np

from dataclasses import dataclass, replace
from typing import Tuple, Sequence, List, Optional

@dataclass(frozen=True)
class Order:
    price: float
    shares: int

@dataclass(frozen=True)
class OrderBook:
    bids: Sequence[Order]
    asks: Sequence[Order]

    def bid_price(self):
        return self.bids[0].price

    def ask_price(self):
        return self.asks[0].price

    def mid_price(self):
        return (self.bid_price() + self.ask_price())/2

    def spread(self):
        return self.bid_price() - self.ask_price()

    def market_depth(self):
        return self.asks[-1].price - self.bids[-1].price

    @staticmethod
    def eat(orders: List[Order], shares: int) -> Tuple[Order, List[Order]]:
        eaten_prices = 0
        eaten_shares = 0
        available_shares = i = 0
        while eaten_shares < shares and i<len(orders):
            order = orders[i]
            available_shares = min(order.shares, shares - eaten_shares)
            eaten_prices += order.price * available_shares
            eaten_shares += available_shares
            i += 1

        eaten_order = Order(eaten_prices, eaten_shares)
        left_shares = orders[i-1].shares - available_shares
        left_orders = ([Order(orders[i-1].price, left_shares)] if left_shares else []) + orders[i:]

        return eaten_order, left_orders

    @staticmethod
    def eat_book(ps_pairs: Sequence[Order],shares: int) -> Tuple[Order, Sequence[Order]]:
        rem_shares: int = shares
        dollars: float = 0.
        for i, d_s in enumerate(ps_pairs):
            this_price: float = d_s.price
            this_shares: int = d_s.shares
            dollars += this_price * min(rem_shares, this_shares)
            if rem_shares < this_shares:
                eaten_order = Order(dollars, shares)
                left_orders = [Order(this_price,this_shares - rem_shares)] + list(ps_pairs[i+1:])
                return eaten_order, left_orders

            else:
                rem_shares -= this_shares

        return Order(dollars, shares - rem_shares), []

    def eat_limit_order(self, price, shares, side) -> Tuple[Order, List[Order]]:
        using_orders = self.bids if side=='sell' else self.asks
        eligibility = lambda x: x.price >= price if side=='sell' else lambda x: x.price <= price
        ground = next((i for i, order in enumerate(using_orders) if not eligibility(order)), len(using_orders))

        # split orders into eligible or ineligible
        eligible_orders = using_orders[:ground]
        ineligible_orders = using_orders[ground:]

        eaten_order, remain_orders = OrderBook.eat(eligible_orders, shares)
        new_countersides = remain_orders + ineligible_orders    # integrate left orders
        remain_shares = shares - eaten_order.shares

        if remain_shares:   # if there is remain limit order
            new_sides = self.asks if side=='sell' else self.bids
            # find this sell order price's position in this-side orders
            position = next((i for i, order in enumerate(new_sides) if eligibility(order)), len(new_sides))
            if position


    def sell_limit_order(self, price: float, shares: int):
        index: Optional[int] = next((i for i, d_s in enumerate(self.bids) if d_s.price < price), None)

        eligible_bids = self.bids if index is None else self.bids[:index]
        ineligible_bids = [] if index is None else self.bids[index:]

        d_s, rem_bids = OrderBook.eat_book(eligible_bids, shares)
        new_bids = list(rem_bids) + list(ineligible_bids)
        rem_shares: int = shares - d_s.shares

        if rem_shares > 0:
            new_asks: List[Order] = list(self.asks)
            index1: Optional[int] = next((i for i, d_s in enumerate(new_asks) if d_s.price >= price), None)
            if index1 is None:
                new_asks.append(Order(price=price, shares=rem_shares))
            elif new_asks[index1].price != price:
                new_asks.insert(index1, Order(price=price,shares=rem_shares))
            else:
                new_asks[index1] = Order(price=price, shares=new_asks[index1].shares + rem_shares)
            return d_s, OrderBook(asks=new_asks,bids=new_bids)
        else:
            return d_s, replace(self,bids=new_bids)


In [46]:
prices = np.random.randint(1, 10, size=10000)
shares = np.random.randint(1, 10, size=10000)
orders = sorted(list(map(lambda x: Order(*x), zip(prices, shares))), key=lambda x: x.price, reverse=True)

In [47]:
orderbook = OrderBook(orders, orders)

In [43]:
orderbook.eat(orders, 50000)

(Order(price=154, shares=20),
 [Order(price=6, shares=1),
  Order(price=6, shares=9),
  Order(price=5, shares=7),
  Order(price=5, shares=6),
  Order(price=3, shares=9),
  Order(price=2, shares=3),
  Order(price=1, shares=5)])

In [44]:
orderbook.eat_book(orders, 50000)

(Order(price=154.0, shares=20),
 [Order(price=6, shares=1),
  Order(price=6, shares=9),
  Order(price=5, shares=7),
  Order(price=5, shares=6),
  Order(price=3, shares=9),
  Order(price=2, shares=3),
  Order(price=1, shares=5)])

In [51]:
import time

def timer(f, args=None, kwargs=None):
    args = [] if args is None else args
    kwargs = dict() if kwargs is None else kwargs
    st = time.time()
    res = f(*args, **kwargs)
    print(f'{f.__name__} ended: {round(time.time()-st, 5)}s')
    return res

In [55]:
res1 = timer(orderbook.eat, [orders, 50000])
res2 = timer(orderbook.eat_book, [orders, 50000])

eat ended: 0.02358s
eat_book ended: 0.01869s


In [63]:
a = [1,2,3]
a.insert(3, 4)
a

[1, 2, 3, 4]